# Construir un Asistente LLM basado en RAG usando Streamlit y Snowflake Cortex Search

*NOTA: Para requisitos previos y otras instrucciones, consulta la [Guía de Inicio Rápido](https://quickstarts.snowflake.com/guide/ask_questions_to_your_own_documents_with_snowflake_cortex_search/index.html#0).*


## Configuración Inicial

Crear una base de datos y un schema.


In [ ]:
-- Crear base de datos y schema
-- CREATE DATABASE IF NOT EXISTS CC_QUICKSTART_CORTEX_SEARCH_DOCS;
-- CREATE SCHEMA IF NOT EXISTS DATA;


## Organizar Documentos y Crear Función de Pre-procesamiento

Paso 1. Los documentos PDF de muestra están incluidos en la carpeta `documentos/`. Son los PDFs originales del repositorio de Snowflake (en inglés) que incluyen especificaciones de productos deportivos.


Paso 2. Crear un Stage con Directory Table donde subirás tus documentos.


In [ ]:
-- Crear stage con cifrado y tabla de directorio habilitada
CREATE OR REPLACE STAGE docs 
ENCRYPTION = (TYPE = 'SNOWFLAKE_SSE') 
DIRECTORY = (ENABLE = true);


Paso 3. Subir documentos PDF a tu área de staging

- Selecciona **Data** en el menú izquierdo
- Haz clic en tu base de datos **CC_QUICKSTART_CORTEX_SEARCH_DOCS**
- Haz clic en tu schema **DATA**
- Haz clic en **Stages** y selecciona **DOCS**
- En la esquina superior derecha haz clic en el botón **+Files**
- Arrastra y suelta los documentos PDF de la carpeta `documentos/`


Paso 4. Verificar que los archivos se han subido correctamente


In [ ]:
-- Listar archivos en el stage
LS @docs;


## Pre-procesar y Etiquetar Documentos

Paso 1. Crear la tabla donde vamos a almacenar los chunks (fragmentos) de cada documento.

Vamos a aprovechar las funciones nativas de procesamiento de documentos de Snowflake para preparar los documentos antes de habilitar Cortex Search. También vamos a usar la función Cortex CLASSIFY_TEXT para etiquetar el tipo de documento que se está procesando, de modo que podamos usar esos metadatos para filtrar búsquedas.


In [ ]:
-- Crear tabla temporal para almacenar el texto extraído de los documentos
CREATE OR REPLACE TEMPORARY TABLE RAW_TEXT AS
SELECT 
    RELATIVE_PATH,
    SIZE,
    FILE_URL,
    BUILD_SCOPED_FILE_URL(@docs, relative_path) AS SCOPED_FILE_URL,
    TO_VARCHAR (
        SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
            '@docs',
            RELATIVE_PATH,
            {'mode': 'LAYOUT'}
        ):content
    ) AS EXTRACTED_LAYOUT 
FROM 
    DIRECTORY('@docs');


In [ ]:
-- Crear tabla para almacenar los chunks de texto
CREATE OR REPLACE TABLE DOCS_CHUNKS_TABLE ( 
    RELATIVE_PATH STRING, -- Ruta relativa al archivo
    SIZE NUMBER(38,0), -- Tamaño del archivo
    FILE_URL STRING, -- URL para el archivo
    SCOPED_FILE_URL STRING, -- URL con scope
    CHUNK STRING, -- Fragmento de texto
    CHUNK_INDEX INTEGER, -- Índice del fragmento
    CATEGORY STRING -- Contendrá la categoría del documento para filtrado
);


Paso 2. Usar la función CORTEX PARSE_DOCUMENT para leer los documentos del área de staging. No es necesario crear embeddings ya que serán gestionados automáticamente por el servicio Cortex Search más adelante.


In [ ]:
-- Insertar los chunks de texto en la tabla
INSERT INTO DOCS_CHUNKS_TABLE (relative_path, size, file_url,
                            scoped_file_url, chunk, chunk_index)
SELECT 
    relative_path, 
    size,
    file_url, 
    scoped_file_url,
    c.value::TEXT AS chunk,
    c.INDEX::INTEGER AS chunk_index
FROM 
    RAW_TEXT,
    LATERAL FLATTEN(
        input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
            EXTRACTED_LAYOUT,
            'markdown',
            1512,  -- Tamaño máximo del chunk
            256,   -- Overlap entre chunks
            ['\n\n', '\n', ' ', '']  -- Separadores
        )
    ) c;


### Etiquetar la categoría del producto

Vamos a usar el poder de los Modelos de Lenguaje Grande y la función [CLASSIFY_TEXT](https://docs.snowflake.com/en/sql-reference/functions/classify_text-snowflake-cortex) para clasificar fácilmente los documentos que estamos ingiriendo en nuestra aplicación RAG. Vamos a pasar el nombre del documento y el primer chunk de texto a la función classify_text.

Primero crearemos una tabla temporal con cada nombre de archivo único y pasaremos ese nombre de archivo y el primer chunk de texto a CLASSIFY_TEXT. La clasificación no es obligatoria para Cortex Search, pero queremos usarla aquí para también demostrar la búsqueda híbrida.


In [ ]:
-- Crear tabla temporal con las categorías de los documentos
CREATE OR REPLACE TEMPORARY TABLE docs_categories AS 
WITH unique_documents AS (
    SELECT
        DISTINCT relative_path, 
        chunk
    FROM
        docs_chunks_table
    WHERE 
        chunk_index = 0  -- Solo usar el primer chunk para clasificación
),
docs_category_cte AS (
    SELECT
        relative_path,
        TRIM(
            SNOWFLAKE.CORTEX.CLASSIFY_TEXT (
                'Título:' || relative_path || ' Contenido:' || chunk, 
                ['Bicicleta', 'Esquí']  -- Categorías posibles
            )['label'], 
            '"'
        ) AS category
    FROM
        unique_documents
)
SELECT
    *
FROM
    docs_category_cte;


Puedes revisar esa tabla para identificar cuántas categorías se han creado y si son correctas:


In [ ]:
-- Verificar las categorías creadas
SELECT category 
FROM docs_categories 
GROUP BY category;


También podemos verificar que la categoría de cada documento sea correcta:


In [ ]:
-- Ver todas las categorías asignadas
SELECT * FROM docs_categories;


Ahora podemos actualizar la tabla con los chunks de texto que serán usados por el servicio Cortex Search para incluir la categoría de cada documento:


In [ ]:
-- Actualizar la tabla principal con las categorías
UPDATE docs_chunks_table 
SET category = docs_categories.category
FROM docs_categories
WHERE docs_chunks_table.relative_path = docs_categories.relative_path;


## Crear Servicio Cortex Search

El siguiente paso es crear el SERVICIO CORTEX SEARCH en la tabla que creamos antes.

- El nombre del servicio es **CC_SEARCH_SERVICE_CS**.
- El servicio usará la columna **chunk** para crear embeddings y realizar recuperación basada en búsqueda de similitud.
- La columna **category** podrá ser usada como filtro.
- Para mantener este servicio actualizado, se usará el warehouse **COMPUTE_WH**. NOTA: Puedes reemplazar el nombre del warehouse con otro al que tengas acceso.
- El servicio se refrescará cada minuto.
- Los datos recuperados contendrán el chunk, relative_path, file_url y category.


In [ ]:
-- Crear el servicio Cortex Search
CREATE OR REPLACE CORTEX SEARCH SERVICE CC_SEARCH_SERVICE_CS
ON chunk  -- Columna sobre la que se crean embeddings
ATTRIBUTES category  -- Columnas que pueden usarse como filtros
WAREHOUSE = COMPUTE_WH  -- Warehouse para mantenimiento
TARGET_LAG = '1 minute'  -- Frecuencia de actualización
AS (
    SELECT 
        chunk,
        chunk_index,
        relative_path,
        file_url,
        category
    FROM docs_chunks_table
);


## Consultar el Servicio Cortex Search

Ahora podemos consultar nuestro servicio usando SQL:


In [ ]:
-- Ejemplo de consulta básica al servicio Cortex Search
SELECT 
    chunk,
    relative_path,
    category
FROM TABLE(
    CC_SEARCH_SERVICE_CS.SEARCH(
        'cuáles son las especificaciones de las bicicletas de carretera?',
        10  -- Número de resultados a devolver
    )
);


In [ ]:
-- Ejemplo de consulta con filtro por categoría
SELECT 
    chunk,
    relative_path,
    category
FROM TABLE(
    CC_SEARCH_SERVICE_CS.SEARCH(
        'qué productos de esquí están disponibles?',
        10,
        {'category': 'Esquí'}  -- Filtrar solo productos de esquí
    )
);


## Procesamiento Automático de Nuevos Documentos

Mantener tu sistema RAG actualizado cuando se agregan, eliminan o actualizan nuevos documentos puede ser tedioso. Snowflake lo hace muy fácil. Por un lado, Cortex Search es un servicio auto-gestionado. Solo necesitamos agregar, eliminar o actualizar filas en la tabla donde se ha habilitado Cortex Search Service y automáticamente el servicio actualizará los índices y creará nuevos embeddings basándose en la frecuencia definida durante la creación del servicio.

Además, podemos usar características de Snowflake como Streams, Tasks y Stored Procedures para procesar automáticamente nuevos archivos a medida que se agregan a Snowflake.

Primero creamos dos streams para el área de staging DOCS. Uno se usará para procesar eliminaciones y otro para procesar inserciones. Los Streams capturan los cambios en la Directory Table usada para el área de staging DOCS. Así podemos rastrear nuevas actualizaciones y eliminaciones:


In [ ]:
-- Crear streams para capturar cambios en el stage
CREATE OR REPLACE STREAM insert_docs_stream ON STAGE docs;
CREATE OR REPLACE STREAM delete_docs_stream ON STAGE docs;


Segundo, vamos a definir un Stored Procedure que procese esos streams para:

- Eliminar de la tabla docs_chunk_table el contenido de archivos que han sido eliminados del área de staging, para que ya no sean relevantes
- Analizar nuevos documentos que han sido agregados al área de staging usando PARSE_DOCUMENT
- Fragmentar el nuevo documento en piezas usando SPLIT_TEXT_RECURSIVE_CHARACTER
- Clasificar los nuevos documentos y actualizar la etiqueta (este paso es opcional, solo para mostrar lo que es posible)

Crear el Stored Procedure:


In [ ]:
-- Crear stored procedure para procesar inserciones y eliminaciones automáticamente
CREATE OR REPLACE PROCEDURE insert_delete_docs_sp()
RETURNS STRING
LANGUAGE SQL
AS
$$
BEGIN
    -- Eliminar documentos que fueron borrados del stage
    DELETE FROM docs_chunks_table
    USING delete_docs_stream
    WHERE docs_chunks_table.RELATIVE_PATH = delete_docs_stream.RELATIVE_PATH
    AND delete_docs_stream.METADATA$ACTION = 'DELETE';

    -- Procesar nuevos documentos agregados
    CREATE OR REPLACE TEMPORARY TABLE RAW_TEXT AS
    SELECT 
        RELATIVE_PATH,
        SIZE,
        FILE_URL,
        BUILD_SCOPED_FILE_URL(@docs, relative_path) AS SCOPED_FILE_URL,
        TO_VARCHAR (
            SNOWFLAKE.CORTEX.PARSE_DOCUMENT (
                '@docs',
                RELATIVE_PATH,
                {'mode': 'LAYOUT'}
            ):content
        ) AS EXTRACTED_LAYOUT 
    FROM 
        insert_docs_stream
    WHERE 
        METADATA$ACTION = 'INSERT';

    -- Insertar nuevos chunks de documentos
    INSERT INTO docs_chunks_table (relative_path, size, file_url,
                            scoped_file_url, chunk, chunk_index)
    SELECT 
        relative_path, 
        size,
        file_url, 
        scoped_file_url,
        c.value::TEXT AS chunk,
        c.INDEX::INTEGER AS chunk_index
    FROM 
        RAW_TEXT,
        LATERAL FLATTEN(
            input => SNOWFLAKE.CORTEX.SPLIT_TEXT_RECURSIVE_CHARACTER (
                EXTRACTED_LAYOUT,
                'markdown',
                1512,
                256,
                ['\n\n', '\n', ' ', '']
            )
        ) c;

    -- Clasificar los nuevos documentos
    CREATE OR REPLACE TEMPORARY TABLE docs_categories AS 
    WITH unique_documents AS (
        SELECT DISTINCT
            d.relative_path, 
            d.chunk
        FROM
            docs_chunks_table d
        INNER JOIN
            RAW_TEXT r
        ON d.relative_path = r.relative_path
        WHERE 
            d.chunk_index = 0
    ),
    docs_category_cte AS (
        SELECT
            relative_path,
            TRIM(
                SNOWFLAKE.CORTEX.CLASSIFY_TEXT (
                    'Título:' || relative_path || ' Contenido:' || chunk, 
                    ['Bicicleta', 'Esquí']
                )['label'], 
                '"'
            ) AS category
        FROM
            unique_documents
    )
    SELECT
        *
    FROM
        docs_category_cte;

    -- Actualizar categorías
    UPDATE docs_chunks_table 
    SET category = docs_categories.category
    FROM docs_categories
    WHERE docs_chunks_table.relative_path = docs_categories.relative_path;
    
    RETURN 'Procesamiento completado con éxito';
END;
$$;


Ahora podemos crear una Task que cada X minutos puede verificar si hay nuevos datos en el stream y tomar una acción. Estamos configurando el schedule a 5 minutos para que puedas seguir la ejecución, pero siéntete libre de reducir el tiempo a 1 minuto si es necesario. Considera qué sería mejor para tu aplicación y con qué frecuencia se actualizan los nuevos documentos.

Definimos:
- **Dónde**: Esto se ejecutará usando el warehouse **COMPUTE_WH**. Por favor cambia al nombre de tu propio Warehouse.
- **Cuándo**: Verificar cada 5 minutos, y ejecutar en caso de que haya nuevos registros en el stream delete_docs_stream (también podríamos usar el otro stream)
- **Qué hacer**: llamar al stored procedure insert_delete_docs_sp()

Ejecuta este código en tu worksheet de Snowflake para crear la task:


In [ ]:
-- Crear task para ejecutar el stored procedure automáticamente
CREATE OR REPLACE TASK insert_delete_docs_task
    WAREHOUSE = COMPUTE_WH
    SCHEDULE = '5 minute'  -- Ejecutar cada 5 minutos
    WHEN SYSTEM$STREAM_HAS_DATA('delete_docs_stream')  -- Solo cuando hay cambios
AS
    CALL insert_delete_docs_sp();

-- Activar la task
ALTER TASK insert_delete_docs_task RESUME;
